# Demo for styringsmøte den 10.03.2025

NB! må kjøre konverting av filer seksjonen først

## Installere pakker


In [5]:
pip install -r requirements.txt


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /usr/local/bin/python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.




## Importering av pakker

In [4]:
import os
import pyarrow.parquet as pq
import geopandas as gpd
import pandas as pd
import datetime
import glob
import json
import time as time_module
from shapely.geometry import box
from IPython.display import display
import folium

## Konstante variabler

In [13]:
ROTMAPPE = "Data"
PARTISJON_MAPPE_HOUR = f"{ROTMAPPE}/partitioned_hour"
PARTISJON_MAPPE_MIN = f"{ROTMAPPE}/partitioned_minutes"

## Konverting av filer

1. Opprett mappestruktur

In [6]:
def opprett_mappestruktur(rotmappe):
    """Oppretter den anbefalte mappestrukturen"""
    mapper = [
        os.path.join(rotmappe, 'raw'),
        os.path.join(rotmappe, 'processed'),
        os.path.join(rotmappe, 'archive')
    ]

    for mappe in mapper:
        os.makedirs(mappe, exist_ok=True)

    return {
        'raw': mapper[0],
        'processed': mapper[1],
        'archive': mapper[2]
    }

2. Sjekk innhold i fil

In [ ]:
def sjekk_metadata_inneholder_geo(schema):
    """Sjekker om parquet-skjemaet inneholder geo-metadata i noen kolonner"""
    for navn in schema.names:
        felt = schema.field(navn)
        if felt.metadata and ('geo' in felt.metadata or b'geo' in felt.metadata):
            return True
    return False

def sjekk_inneholder_geometrikolonne(filsti):
    """Sjekker om filen inneholder en geometrikolonne ved å lese den med geopandas"""
    gdf = gpd.read_parquet(filsti)
    return hasattr(gdf, 'geometry') and gdf.geometry.name in gdf.columns

def er_geoparquet(filsti):
    """Sjekker om en parquet-fil er en GeoParquet-fil ved å undersøke metadata og innhold"""
    # Hvis filen ikke kan leses som parquet, er den ikke en geoparquet
    try:
        schema = pq.read_schema(filsti)
    except:
        return False

    # Sjekk metadata
    if sjekk_metadata_inneholder_geo(schema):
        return True

    # Sjekk geometrikolonne i geopandas
    try:
        return sjekk_inneholder_geometrikolonne(filsti)
    except:
        return False

3. Konverter fil til geoparquet

In [4]:
def lag_målfilsti(kildefilsti, målmappesti=None):
    """Lager målfilsti basert på kildesti og evt. målmappe"""
    base_navn = os.path.basename(kildefilsti)
    base_navn_uten_endelse = os.path.splitext(base_navn)[0]

    if målmappesti is None:
        # Hvis ingen målmappe er angitt, bruk samme mappe med _geo.parquet
        dir_navn = os.path.dirname(kildefilsti)
        return os.path.join(dir_navn, f"{base_navn_uten_endelse}_geo.parquet")
    else:
        # Hvis målmappe er angitt, plasser filen der
        return os.path.join(målmappesti, f"{base_navn_uten_endelse}.parquet")

def opprett_mappe(filsti):
    """Sørger for at mappen for filstien eksisterer"""
    os.makedirs(os.path.dirname(filsti), exist_ok=True)

def konverter_parquet(filsti):
    """Konverterer parquet-fil til GeoDataFrame"""
    df = pd.read_parquet(filsti)

    # Sjekk etter lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        return gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df.longitude, df.latitude),
            crs="EPSG:4326"
        )

    # Sjekk etter potensielle geometrikolonner
    geom_kolonner = [col for col in df.columns if 'geom' in col.lower()
                   or 'coord' in col.lower()
                   or 'point' in col.lower()
                   or 'polygon' in col.lower()
                   or 'linestring' in col.lower()
                   or 'wkt' in col.lower()]

    for col in geom_kolonner:
        if df[col].dtype == 'object':
            try:
                # Prøv å konvertere fra WKT-format
                geom = df[col].apply(wkt.loads)
                return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
            except:
                continue

    return None

def konverter_csv(filsti):
    """Konverterer CSV-fil til GeoDataFrame"""
    df = pd.read_csv(filsti)

    # Sjekk for vanlige lat/long kolonnenavn
    lat_kolonner = ['latitude', 'lat', 'y', 'breddegrad']
    lon_kolonner = ['longitude', 'long', 'lon', 'x', 'lengdegrad']

    lat_col = next((col for col in lat_kolonner if col in df.columns), None)
    lon_col = next((col for col in lon_kolonner if col in df.columns), None)

    if lat_col and lon_col:
        return gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
            crs="EPSG:4326"
        )

    return None

def konverter_fil_basert_på_type(filsti, filendelse):
    """Konverterer fil til GeoDataFrame basert på filendelse"""
    if filendelse == '.parquet':
        return konverter_parquet(filsti)

    elif filendelse in ['.geojson', '.json']:
        return gpd.read_file(filsti)

    elif filendelse == '.shp':
        return gpd.read_file(filsti)

    elif filendelse == '.gpkg':
        return gpd.read_file(filsti, driver='GPKG')

    elif filendelse == '.csv':
        return konverter_csv(filsti)

    return None

def konverter_til_geoparquet(kildefilsti, målmappesti=None):
    """Konverterer ulike geografiske formater til GeoParquet-format"""
    # Få filendelse
    filendelse = os.path.splitext(kildefilsti)[1].lower()

    # Bestem målfilsti
    målfilsti = lag_målfilsti(kildefilsti, målmappesti)

    # Sørg for at målmappen eksisterer
    opprett_mappe(målfilsti)

    try:
        # Konverter filen basert på filtype
        gdf = konverter_fil_basert_på_type(kildefilsti, filendelse)

        # Hvis vi har en gyldig geodataframe, lagre som GeoParquet
        if gdf is not None:
            gdf.to_parquet(målfilsti)
            return målfilsti
    except Exception as e:
        print(f"Feil ved konvertering av {kildefilsti}: {e}")

    return False

4. Let etter og identifiserer geografiske datafiler som ennå ikke er blitt konvertert til GeoParquet-format

In [5]:
def hent_støttede_formater():
    """Returnerer liste over støttede filformater"""
    return ['.parquet', '.geojson', '.json', '.shp', '.gpkg', '.csv']

def hent_konverterte_filer(prosessert_mappesti):
    """Henter ut alle allerede konverterte filer"""
    konverterte_filer = set()
    if not os.path.exists(prosessert_mappesti):
        return konverterte_filer

    for root, _, filer in os.walk(prosessert_mappesti):
        for fil in filer:
            if fil.endswith('.parquet'):
                base_navn = os.path.splitext(fil)[0]
                konverterte_filer.add(base_navn)

    return konverterte_filer

def les_konverteringslogg(konverteringslogg_sti):
    """Leser konverteringsloggen fra fil"""
    if not konverteringslogg_sti or not os.path.exists(konverteringslogg_sti):
        return {}

    try:
        with open(konverteringslogg_sti, 'r') as f:
            return json.load(f)
    except:
        return {}

def er_fil_gyldig_for_konvertering(filsti, base_navn, filendelse, støttede_formater):
    """Sjekker om en fil er gyldig for konvertering"""
    # Sjekk om filen har et støttet format
    if filendelse not in støttede_formater:
        return False

    # Parquet-filer trenger ekstra sjekk
    if filendelse == '.parquet':
        # Ikke ta med filer som allerede er GeoParquet
        if er_geoparquet(filsti):
            return False
        # Ikke ta med filer som allerede har _geo suffix
        if base_navn.endswith('_geo'):
            return False

    return True

def trenger_fil_konvertering(filsti, base_navn, konverterte_filer, konverterte_logger):
    """Sjekker om en fil trenger konvertering basert på konverteringshistorikk"""
    # Sjekk om filen allerede er konvertert
    if base_navn in konverterte_filer:
        return False

    # Sjekk om filen har endret seg siden sist konvertering
    if base_navn in konverterte_logger:
        sist_endret_tid = os.path.getmtime(filsti)
        sist_konvertert_tid = konverterte_logger[base_navn].get('tidspunkt', 0)
        if sist_endret_tid <= sist_konvertert_tid:
            return False

    return True

def finn_nye_filer(rå_mappesti, prosessert_mappesti, konverteringslogg_sti=None):
    """Finner filer som ikke har blitt konvertert ennå"""
    støttede_formater = hent_støttede_formater()
    konverterte_filer = hent_konverterte_filer(prosessert_mappesti)
    konverterte_logger = les_konverteringslogg(konverteringslogg_sti)

    nye_filer = []

    for root, _, filer in os.walk(rå_mappesti):
        for fil in filer:
            filsti = os.path.join(root, fil)
            filendelse = os.path.splitext(fil)[1].lower()
            base_navn = os.path.splitext(fil)[0]

            # Sjekk om filen er gyldig for konvertering
            if not er_fil_gyldig_for_konvertering(filsti, base_navn, filendelse, støttede_formater):
                continue

            # Sjekk om filen trenger konvertering
            if trenger_fil_konvertering(filsti, base_navn, konverterte_filer, konverterte_logger):
                nye_filer.append(filsti)

    return nye_filer

5. Registrer informasjon om nylig konverterte geografiske datafiler i en loggfil

In [7]:
def les_eksisterende_logg(konverteringslogg_sti):
    """Leser inn eksisterende konverteringslogg"""
    if not os.path.exists(konverteringslogg_sti):
        return {}

    try:
        with open(konverteringslogg_sti, 'r') as f:
            return json.load(f)
    except:
        return {}

def lag_logginnslag(fil, målfilsti):
    """Lager et nytt logginnslag for en konvertert fil"""
    base_navn = os.path.splitext(os.path.basename(fil))[0]
    nå = datetime.datetime.now()

    return {
        base_navn: {
            'kildesti': fil,
            'målsti': målfilsti,
            'tidspunkt': nå.timestamp(),
            'dato': nå.strftime('%Y-%m-%d %H:%M:%S')
        }
    }

def lagre_logg(konverteringslogg_sti, loggdata):
    """Lagrer konverteringslogg til fil"""
    with open(konverteringslogg_sti, 'w') as f:
        json.dump(loggdata, f, indent=2)

def oppdater_konverteringslogg(konverteringslogg_sti, fil, målfilsti):
    """Oppdaterer konverteringsloggen med ny filinfo"""
    # Last inn eksisterende logg
    konverterte_logger = les_eksisterende_logg(konverteringslogg_sti)

    # Lag og legg til nytt logginnslag
    nytt_innslag = lag_logginnslag(fil, målfilsti)
    konverterte_logger.update(nytt_innslag)

    # Lagre oppdatert logg
    lagre_logg(konverteringslogg_sti, konverterte_logger)

6. Ser gjennom en mappestruktur, identifiserer geografiske datafiler, konverterer dem til GeoParquet-format hvis de ikke allerede er det, og lager en detaljert rapport over resultatet.

In [6]:
def valider_rotmappe(rotmappesti):
    """Validerer at rotmappen eksisterer"""
    if not os.path.exists(rotmappesti):
        print(f"FEIL: Katalogen '{rotmappesti}' eksisterer ikke.")
        return False
    return True

def oppsett_mappestier(rotmappesti, opprett_struktur):
    """Setter opp og returnerer nødvendige mappestier"""
    mappestruktur = None
    rå_mappesti = rotmappesti
    prosessert_mappesti = rotmappesti

    # Opprett mappestruktur hvis ønsket
    if opprett_struktur:
        mappestruktur = opprett_mappestruktur(rotmappesti)
        rå_mappesti = mappestruktur['raw']
        prosessert_mappesti = mappestruktur['processed']
        print(f"Opprettet mappestruktur:\n- Raw: {rå_mappesti}\n- Processed: {prosessert_mappesti}")

    return {
        "mappestruktur": mappestruktur,
        "rå_mappesti": rå_mappesti,
        "prosessert_mappesti": prosessert_mappesti,
        "konverteringslogg_sti": os.path.join(rotmappesti, 'conversion_log.json')
    }

def konverter_filer(nye_filer, prosessert_mappesti, konverteringslogg_sti):
    """Konverterer filer og oppdaterer logg"""
    resultater = {
        "konvertert": [],
        "feilet": []
    }

    for filsti in nye_filer:
        # Konverter filen
        målfilsti = konverter_til_geoparquet(filsti, prosessert_mappesti)

        if målfilsti:
            resultater["konvertert"].append(filsti)
            oppdater_konverteringslogg(konverteringslogg_sti, filsti, målfilsti)
        else:
            resultater["feilet"].append(filsti)

    return resultater

def finn_eksisterende_geoparquet(rå_mappesti, alle_filer):
    """Finner eksisterende GeoParquet-filer"""
    allerede_geoparquet = []

    for root, _, filer in os.walk(rå_mappesti):
        for fil in filer:
            if fil.endswith('.parquet'):
                filsti = os.path.join(root, fil)
                if er_geoparquet(filsti) and filsti not in alle_filer:
                    allerede_geoparquet.append(filsti)

    return allerede_geoparquet

def skriv_oppsummering(rotmappesti, mappestier, resultater):
    """Skriver oppsummering av konverteringsprosessen"""
    print("\n" + "="*50)
    print("OPPSUMMERING")
    print("="*50)
    print(f"Katalog: {rotmappesti}")

    if mappestier["mappestruktur"]:
        print(f"Mappestruktur: Raw={mappestier['rå_mappesti']}, Processed={mappestier['prosessert_mappesti']}")

    totalt_antall = len(resultater["alle_filer"])
    antall_allerede_geo = len(resultater["allerede_geoparquet"])
    antall_konvertert = len(resultater["konvertert"])
    antall_feilet = len(resultater["feilet"])

    print(f"Totalt antall geografiske filer funnet: {totalt_antall}")
    print(f"Antall parquet-filer som allerede var GeoParquet: {antall_allerede_geo}")
    print(f"Antall filer konvertert til GeoParquet: {antall_konvertert}")
    print(f"Antall filer som ikke kunne konverteres: {antall_feilet}")

    if resultater["konvertert"]:
        print("\nKonverterte filer:")
        for fil in resultater["konvertert"]:
            print(f"- {fil}")

    if resultater["feilet"]:
        print("\nFiler som ikke kunne konverteres:")
        for fil in resultater["feilet"]:
            print(f"- {fil}")

def behandle_alle_filer_i_mappe(rotmappesti, opprett_struktur=True):
    """Behandler alle geografiske filer i angitt mappe og konverterer til GeoParquet."""
    # Validering
    if not valider_rotmappe(rotmappesti):
        return None

    # Oppsett av mappestruktur
    mappestier = oppsett_mappestier(rotmappesti, opprett_struktur)

    # Finn nye filer som trenger konvertering
    nye_filer = finn_nye_filer(
        mappestier["rå_mappesti"],
        mappestier["prosessert_mappesti"],
        mappestier["konverteringslogg_sti"]
    )

    # Initialiser resultater
    resultater = {
        "allerede_geoparquet": [],
        "konvertert": [],
        "feilet": [],
        "alle_filer": nye_filer.copy()
    }

    # Konverter filer
    konverteringsresultater = konverter_filer(
        nye_filer,
        mappestier["prosessert_mappesti"],
        mappestier["konverteringslogg_sti"]
    )
    resultater["konvertert"] = konverteringsresultater["konvertert"]
    resultater["feilet"] = konverteringsresultater["feilet"]

    # Finn allerede eksisterende GeoParquet-filer
    resultater["allerede_geoparquet"] = finn_eksisterende_geoparquet(
        mappestier["rå_mappesti"],
        resultater["alle_filer"]
    )
    resultater["alle_filer"].extend(resultater["allerede_geoparquet"])

    # Skriv oppsummering
    skriv_oppsummering(rotmappesti, mappestier, resultater)

    return resultater

# Angi rotmappen som skal behandles
resultater = behandle_alle_filer_i_mappe(ROTMAPPE, opprett_struktur=True)

## Filtering
Leser, filtrere og analysere geografiske data fra en GeoParquet-fil basert på gitte kriterier

In [7]:
# MÅ HENTE RIKTIG FIL!

# Les geoparquet-filen
fil_sti = f"{ROTMAPPE}/raw/hais_2025-01-01.snappy.parquet"
gdf = gpd.read_parquet(fil_sti)

# Filtrering basert på attributt/kolonne
if 'date_time_utc' in gdf.columns:
    filtrert_på_dato = gdf[gdf['date_time_utc'] >= '2025-01-01 12:00:00']
    print(f"Antall rader etter datofiltrering: {len(filtrert_på_dato)}")
    display(filtrert_på_dato.head())

# Filtrering basert på geometri (f.eks. et område)
område = box(5.0, 60.0, 11.0, 60.0)
innenfor_område = gdf[gdf.geometry.intersects(område)]
print(f"Antall rader innenfor det definerte området: {len(innenfor_område)}")
display(innenfor_område.head())

# Kombinert filtrering med flere kriterier
if 'attributt' in gdf.columns:
    kombinert_filter = gdf[(gdf.geometry.intersects(område)) & (gdf['attributt'] > 10)]
    print(f"Antall rader etter kombinert filtrering: {len(kombinert_filter)}")
    display(kombinert_filter.head())

NameError: name 'ROTMAPPE' is not defined

1. Funksjoner

In [ ]:
def les_geoparquet(fil_sti):
    """Leser en GeoParquet-fil og returnerer en GeoDataFrame"""
    return gpd.read_parquet(fil_sti)

def filtrer_på_dato(gdf, dato_kolonne, start_dato):
    """Filtrerer GeoDataFrame basert på dato"""
    if dato_kolonne not in gdf.columns:
        print(f"ADVARSEL: Kolonnen '{dato_kolonne}' finnes ikke i datasettet")
        return gdf

    filtrert = gdf[gdf[dato_kolonne] >= start_dato]
    print(f"Antall rader etter datofiltrering: {len(filtrert)}")
    return filtrert

def filtrer_på_område(gdf, min_x, min_y, max_x, max_y):
    """Filtrerer GeoDataFrame basert på et geografisk område"""
    område = box(min_x, min_y, max_x, max_y)
    filtrert = gdf[gdf.geometry.intersects(område)]
    print(f"Antall rader innenfor det definerte området: {len(filtrert)}")
    return filtrert

def filtrer_på_attributt(gdf, attributt, verdi, operator='større'):
    """Filtrerer GeoDataFrame basert på et attributt"""
    if attributt not in gdf.columns:
        print(f"ADVARSEL: Kolonnen '{attributt}' finnes ikke i datasettet")
        return gdf

    if operator == 'større':
        filtrert = gdf[gdf[attributt] > verdi]
    elif operator == 'mindre':
        filtrert = gdf[gdf[attributt] < verdi]
    elif operator == 'lik':
        filtrert = gdf[gdf[attributt] == verdi]
    else:
        print(f"ADVARSEL: Ukjent operator '{operator}'")
        return gdf

    print(f"Antall rader etter attributtfiltrering: {len(filtrert)}")
    return filtrert

def kombiner_filtre(gdf, område_filter=None, dato_filter=None, attributt_filter=None):
    """Kombinerer flere filtre på en GeoDataFrame"""
    resultat = gdf.copy()

    if område_filter:
        min_x, min_y, max_x, max_y = område_filter
        område = box(min_x, min_y, max_x, max_y)
        resultat = resultat[resultat.geometry.intersects(område)]

    if dato_filter:
        kolonne, start_dato = dato_filter
        if kolonne in resultat.columns:
            resultat = resultat[resultat[kolonne] >= start_dato]

    if attributt_filter:
        kolonne, verdi, operator = attributt_filter
        if kolonne in resultat.columns:
            if operator == 'større':
                resultat = resultat[resultat[kolonne] > verdi]
            elif operator == 'mindre':
                resultat = resultat[resultat[kolonne] < verdi]
            elif operator == 'lik':
                resultat = resultat[resultat[kolonne] == verdi]

    print(f"Antall rader etter kombinert filtrering: {len(resultat)}")
    return resultat

# Eksempel på bruk:
def analyser_ais_data():
    # Konstanter
    ROTMAPPE = "/sti/til/data"
    # MÅ HENTE RIKTIG FIL!
    fil_sti = f"{ROTMAPPE}/raw/hais_2025-01-01.snappy.parquet"

    # Les data
    gdf = les_geoparquet(fil_sti)
    print(f"Totalt antall rader i datasettet: {len(gdf)}")

    # Enkle filtreringer
    gdf_dato = filtrer_på_dato(gdf, 'date_time_utc', '2025-01-01 12:00:00')
    gdf_område = filtrer_på_område(gdf, 5.0, 60.0, 11.0, 60.0)

    # Kombinert filtrering
    gdf_kombinert = kombiner_filtre(
        gdf,
        område_filter=(5.0, 60.0, 11.0, 60.0),
        attributt_filter=('attributt', 10, 'større')
    )

    return {
        'original': gdf,
        'dato_filtrert': gdf_dato,
        'område_filtrert': gdf_område,
        'kombinert_filtrert': gdf_kombinert
    }

2. Innledende analyse av datasett

In [ ]:
# Oppsett og lesing av data
ROTMAPPE = "/sti/til/data"
fil_sti = f"{ROTMAPPE}/raw/hais_2025-01-01.snappy.parquet"
gdf = les_geoparquet(fil_sti)

# Vise grunnleggende datasettinfo
print(f"Totalt antall rader i datasettet: {len(gdf)}")
print(f"Kolonner i datasettet: {list(gdf.columns)}")
gdf.head()

3. Tidsbasert analyse

In [ ]:
# Filtrere på dato
gdf_dato = filtrer_på_dato(gdf, 'date_time_utc', '2025-01-01 12:00:00')
gdf_dato.head()

3. Geografisk analyse

In [ ]:
# Filtrere på geografisk område
gdf_område = filtrer_på_område(gdf, 5.0, 60.0, 11.0, 60.0)

# Kanskje vise et kart med området og punktene
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(ax=ax, color='gray', alpha=0.3, markersize=5)
gdf_område.plot(ax=ax, color='red', markersize=5)
ax.set_title('Geografisk filtrering')
plt.show()

4. Kombinert analyse

In [ ]:
# Kombinert filtrering for mer spesifikke analyseformål
gdf_kombinert = kombiner_filtre(
    gdf,
    område_filter=(5.0, 60.0, 11.0, 60.0),
    attributt_filter=('attributt', 10, 'større')
)
gdf_kombinert.head()

## Partisjonering
Partisjonerer geografiske skipstrafikkdata basert på tid og skipstype

1. Konstanter

In [ ]:
ROTMAPPE = "./data"
PARTISJON_MAPPE_HOUR = os.path.join(ROTMAPPE, "partisjonert_time")
PARTISJON_MAPPE_MIN = os.path.join(ROTMAPPE, "partisjonert_10min")

2. Funksjoner

In [ ]:
def opprett_mappe(mappesti):
    """Oppretter en mappe hvis den ikke eksisterer"""
    os.makedirs(mappesti, exist_ok=True)
    return mappesti

def lag_time_partisjon(data, målfolder):
    """Partisjonerer geodata basert på time og skipstype"""
    # Lag en kopi for å unngå SettingWithCopyWarning
    data_for_partisjonering = data.copy()

    # Bruk .loc for å legge til time-kolonnen
    data_for_partisjonering.loc[:, 'time'] = data_for_partisjonering['date_time_utc'].dt.hour

    # Opprett hovedmappen
    opprett_mappe(målfolder)

    antall_filer = 0
    antall_rader = 0

    # Partisjonering basert på time og ship_type
    for time, gruppe_time in data_for_partisjonering.groupby('time'):
        time_mappe = os.path.join(målfolder, f"time={time:02d}")
        opprett_mappe(time_mappe)

        for ship_type, gruppe_final in gruppe_time.groupby('ship_type'):
            skip_mappe = os.path.join(time_mappe, f"ship_type={ship_type}")
            opprett_mappe(skip_mappe)

            # Bruk en kopi av dataene
            data_å_lagre = gruppe_final.copy()

            # Fjern partisjoneringskolonnen før lagring
            fil_sti = os.path.join(skip_mappe, f"data.parquet")
            data_å_lagre.drop('time', axis=1).to_parquet(fil_sti)

            print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")
            antall_filer += 1
            antall_rader += len(gruppe_final)

    return {'antall_filer': antall_filer, 'antall_rader': antall_rader}

def lag_10min_partisjon(data, målfolder):
    """Partisjonerer geodata basert på 10-minutters intervaller og skipstype"""
    # Lag en kopi for å unngå SettingWithCopyWarning
    data_for_partisjonering = data.copy()

    # Bruk .loc for å legge til 10-minutters intervall kolonne
    # Dette gir intervaller 0-143 for hele dagen (144 intervaller på 10 minutter)
    data_for_partisjonering.loc[:, 'minuttgruppe'] = (
        data_for_partisjonering['date_time_utc'].dt.hour * 6 +
        data_for_partisjonering['date_time_utc'].dt.minute // 10
    )

    # Opprett hovedmappen
    opprett_mappe(målfolder)

    antall_filer = 0
    antall_rader = 0

    # Partisjonering basert på minuttgruppe og ship_type
    for minuttgruppe, gruppe_minutt in data_for_partisjonering.groupby('minuttgruppe'):
        # Konverter minuttgruppe til time og minutt for mappe-strukturen
        time = minuttgruppe // 6
        minutt = (minuttgruppe % 6) * 10

        # Lag en lesbar mappestruktur (time_minutt=HH_MM)
        minutt_mappe = os.path.join(målfolder, f"time_minutt={time:02d}_{minutt:02d}")
        opprett_mappe(minutt_mappe)

        for ship_type, gruppe_final in gruppe_minutt.groupby('ship_type'):
            skip_mappe = os.path.join(minutt_mappe, f"ship_type={ship_type}")
            opprett_mappe(skip_mappe)

            # Bruk en kopi av dataene
            data_å_lagre = gruppe_final.copy()

            # Fjern partisjoneringskolonnen før lagring
            fil_sti = os.path.join(skip_mappe, f"data.parquet")
            data_å_lagre.drop('minuttgruppe', axis=1).to_parquet(fil_sti)

            print(f"Skrevet {len(gruppe_final)} rader til {fil_sti}")
            antall_filer += 1
            antall_rader += len(gruppe_final)

    return {'antall_filer': antall_filer, 'antall_rader': antall_rader}

def les_partisjonerte_data_for_måned(rotmappesti, ønsket_måned):
    """Leser partisjonerte data for en spesifikk måned"""
    måned_sti = os.path.join(rotmappesti, f"år_måned={ønsket_måned}")

    if not os.path.exists(måned_sti):
        print(f"Ingen mappe funnet for {ønsket_måned}")
        return None

    print(f"Leser data for {ønsket_måned}...")
    # Finn alle geoparquet-filer i denne månedens mappe (inkludert undermapper)
    filer = glob.glob(os.path.join(måned_sti, "**/*.parquet"), recursive=True)

    # Les og kombiner alle filene
    dataframes = []
    for fil in filer:
        gdf = gpd.read_parquet(fil)
        dataframes.append(gdf)

    if not dataframes:
        print(f"Ingen data funnet for {ønsket_måned}")
        return None

    månedsdata = pd.concat(dataframes)
    print(f"Hentet {len(månedsdata)} rader for {ønsket_måned}")
    return månedsdata

def les_partisjonerte_data_for_skipstype(rotmappesti, ønsket_skipstype):
    """Leser partisjonerte data for en spesifikk skipstype"""
    skipstype_stier = glob.glob(os.path.join(rotmappesti, f"**/ship_type={ønsket_skipstype}/*.parquet"), recursive=True)

    if not skipstype_stier:
        print(f"Ingen data funnet for skipstype {ønsket_skipstype}")
        return None

    print(f"Leser data for skipstype {ønsket_skipstype}...")
    skipstype_dataframes = []
    for fil in skipstype_stier:
        gdf = gpd.read_parquet(fil)
        skipstype_dataframes.append(gdf)

    skipstype_data = pd.concat(skipstype_dataframes)
    print(f"Hentet {len(skipstype_data)} rader for skipstype {ønsket_skipstype}")
    return skipstype_data

3. Dataforberedelse

In [ ]:
# Les inn datasettet
fil_sti = f"{ROTMAPPE}/raw/hais_2025-01-01.snappy.parquet"
gdf = gpd.read_parquet(fil_sti)
print(f"Totalt antall rader i datasettet: {len(gdf)}")

# Filtrer data til et bestemt geografisk område
område = box(5.0, 60.0, 11.0, 60.0)
innenfor_område = gdf[gdf.geometry.intersects(område)]
print(f"Antall rader innenfor området: {len(innenfor_område)}")

# Vis første rader
display(innenfor_område.head())

4. Timebasert partisjonering

In [ ]:
# Partisjonering basert på time
resultat_time = lag_time_partisjon(innenfor_område, PARTISJON_MAPPE_HOUR)

# Oppsummering
print(f"\nTimebasert partisjonering fullført:")
print(f"- Antall filer opprettet: {resultat_time['antall_filer']}")
print(f"- Totalt antall rader lagret: {resultat_time['antall_rader']}")

5. 10-minutters partisjonering

In [ ]:
# Partisjonering basert på 10-minutters intervaller
resultat_10min = lag_10min_partisjon(innenfor_område, PARTISJON_MAPPE_MIN)

# Oppsummering
print(f"\n10-minutters partisjonering fullført:")
print(f"- Antall filer opprettet: {resultat_10min['antall_filer']}")
print(f"- Totalt antall rader lagret: {resultat_10min['antall_rader']}")

6. Les partisjonert data for en spesifikk måned

In [ ]:
# Definer ønsket måned
ønsket_måned = "2025-01"

# Les data for den valgte måneden
månedsdata = les_partisjonerte_data_for_måned(PARTISJON_MAPPE_HOUR, ønsket_måned)

# Vis data hvis tilgjengelig
if månedsdata is not None:
    display(månedsdata.head())

    # Lag en enkel visualisering
    fig, ax = plt.subplots(figsize=(10, 8))
    månedsdata.plot(ax=ax, column='ship_type', cmap='viridis', legend=True)
    ax.set_title(f'Skipstrafikk i {ønsket_måned}')
    plt.show()

7. Les partisjonerte data for en spesifikk skipstype

In [ ]:
# Definer ønsket skipstype
ønsket_skipstype = 70

# Les data for den valgte skipstypen
skipstypedata = les_partisjonerte_data_for_skipstype(PARTISJON_MAPPE_HOUR, ønsket_skipstype)

# Vis data hvis tilgjengelig
if skipstypedata is not None:
    display(skipstypedata.head())

    # Visualiser dataene
    fig, ax = plt.subplots(figsize=(10, 8))
    skipstypedata.plot(ax=ax, color='red', alpha=0.5)
    ax.set_title(f'Posisjon for skipstype {ønsket_skipstype}')
    plt.show()

    # Lagre resultatene til en fil
    # skipstypedata.to_parquet(f"{ROTMAPPE}/resultater/skipstype_{ønsket_skipstype}.parquet")

## Simulering av datastrømming

In [ ]:
# Simulert datastrømming med visualisering

# Finn alle minutt-mapper sortert kronologisk
alle_intervaller = sorted([d for d in os.listdir(PARTISJON_MAPPE_MIN) if d.startswith("time_minutt=")])

# Funksjon for å lage et kart med skipsbevegelser
def vis_skip_på_kart(data, tidspunkt):
    # Konverter til WGS84 koordinatsystem hvis nødvendig
    if data.crs and data.crs != "EPSG:4326":
        data = data.to_crs("EPSG:4326")

    # Beregn senterpunkt for kartet
    midpoint_lat = data.geometry.centroid.y.mean()
    midpoint_lon = data.geometry.centroid.x.mean()

    # Opprett et kart sentrert på dataenes midtpunkt
    m = folium.Map(location=[midpoint_lat, midpoint_lon], zoom_start=10)

    # Legg til en tittel
    tittel_html = f'''
    <h3 align="center" style="font-size:16px"><b>Skipsbevegelser: {tidspunkt}</b></h3>
    '''
    m.get_root().html.add_child(folium.Element(tittel_html))

    # Fargekoding basert på skipstype
    ship_type_colors = {
        30: 'blue',      # Fiskefartøy
        31: 'green',     # Slepebåt
        52: 'red',       # Passasjerskip
        60: 'purple',    # Passasjerskip
        70: 'orange',    # Lasteskip
        80: 'darkblue',  # Tankskip
        # Legg til flere skipstyper etter behov
    }

    # Legg til hvert skip på kartet
    for idx, row in data.iterrows():
        # Bestem farge basert på skipstype
        color = ship_type_colors.get(row['ship_type'], 'gray')

        # Hent koordinater
        coords = (row.geometry.y, row.geometry.x)

        # Lag popup-info
        popup_text = f"""
        <b>Skip:</b> {row['ship_name']}<br>
        <b>MMSI:</b> {row['mmsi']}<br>
        <b>Type:</b> {row['ship_type']}<br>
        <b>Hastighet:</b> {row['speed_over_ground']} knop<br>
        <b>Kurs:</b> {row['course_over_ground']}°<br>
        <b>Tidspunkt:</b> {row['date_time_utc']}
        """

        # Legg til markør med retningspil
        folium.Marker(
            coords,
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color=color, icon='ship', prefix='fa'),
            tooltip=f"{row['ship_name']} ({row['mmsi']})"
        ).add_to(m)

        # Tegn retningspil hvis kurs er tilgjengelig
        if not pd.isna(row['course_over_ground']):
            folium.RegularPolygonMarker(
                coords,
                fill_color=color,
                number_of_sides=3,
                radius=8,
                rotation=row['course_over_ground'],
                fill_opacity=0.6,
                color='black',
                weight=1
            ).add_to(m)

    # Legg til tegnforklaring
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; right: 50px; width: 150px; height: 160px;
                border:2px solid grey; z-index:9999; font-size:12px;
                background-color: white; padding: 10px;
                border-radius: 5px;">
    <b>Skipstyper:</b><br>
    '''

    for ship_type, color in ship_type_colors.items():
        legend_html += f'<i class="fa fa-ship" style="color:{color}"></i> Type {ship_type}<br>'

    legend_html += '</div>'
    m.get_root().html.add_child(folium.Element(legend_html))

    # Returner kartet
    return m

# Simuler datastrømming med visualisering
print("\nSimulerer datastrømming med visualisering...")
for intervall_dir in alle_intervaller:
    # Hent time og minutt fra mappenavnet
    match = re.search(r"time_minutt=(\d{2})_(\d{2})", intervall_dir)
    if match:
        time, minutt = match.groups()
        tidspunkt = f"{time}:{minutt}"

        intervall_sti = os.path.join(PARTISJON_MAPPE_MIN, intervall_dir)
        print(f"\nProsesserer data for {tidspunkt}...")

        # Finn alle geoparquet-filer for dette intervallet
        filer = glob.glob(os.path.join(intervall_sti, "**/*.parquet"), recursive=True)

        # Les og bearbeid hver fil
        intervall_data = []
        for fil in filer:
            gdf = gpd.read_parquet(fil)
            intervall_data.append(gdf)

        if intervall_data:
            samlet_data = pd.concat(intervall_data)
            print(f"Kl. {tidspunkt}: Lastet {len(samlet_data)} rader fra {len(filer)} filer")

            # Vis statistikker
            print(f"  Gjennomsnittlig hastighet (SOG): {samlet_data['speed_over_ground'].mean():.2f}")
            print(f"  Antall unike skip: {samlet_data['mmsi'].nunique()}")

            # Opprett og vis kart
            kart = vis_skip_på_kart(samlet_data, f"Kl. {tidspunkt}")

            # Lagre kartet til en HTML-fil
            # kart_filnavn = f"kart_{time}_{minutt}.html"
            # kart.save(kart_filnavn)
            # print(f"  Kart lagret som {kart_filnavn}")

            display(kart)

        else:
            print(f"Ingen data funnet for kl. {tidspunkt}")

        # Simuler tid mellom intervaller
        time_module.sleep(0.5)

print("\nDatastrømming komplett!")

1. Konstanter

In [ ]:
PARTISJON_MAPPE_MIN = "din_partisjon_mappe_her"

2. Funksjoner

In [ ]:
def finn_minutt_intervaller(hovedmappe):
    """Finn alle minutt-mapper sortert kronologisk"""
    return sorted([d for d in os.listdir(hovedmappe) if d.startswith("time_minutt=")])

def les_data_for_intervall(intervall_sti):
    """Les og kombiner alle geoparquet-filer for et gitt intervall"""
    # Finn alle geoparquet-filer for dette intervallet
    filer = glob.glob(os.path.join(intervall_sti, "**/*.parquet"), recursive=True)

    # Les og bearbeid hver fil
    intervall_data = []
    for fil in filer:
        gdf = gpd.read_parquet(fil)
        intervall_data.append(gdf)

    # Kombiner data hvis noen filer ble funnet
    if intervall_data:
        return pd.concat(intervall_data), len(filer)
    else:
        return None, 0

def beregn_statistikk(data):
    """Beregn og returner statistikk for datasettet"""
    return {
        "gjennomsnittlig_hastighet": data['speed_over_ground'].mean(),
        "antall_unike_skip": data['mmsi'].nunique(),
        "totalt_antall_rader": len(data)
    }

def vis_skip_på_kart(data, tidspunkt):
    """Opprett et kart med skipsbevegelser"""
    # Konverter til WGS84 koordinatsystem hvis nødvendig
    if data.crs and data.crs != "EPSG:4326":
        data = data.to_crs("EPSG:4326")

    # Beregn senterpunkt for kartet
    midpoint_lat = data.geometry.centroid.y.mean()
    midpoint_lon = data.geometry.centroid.x.mean()

    # Opprett et kart sentrert på dataenes midtpunkt
    m = folium.Map(location=[midpoint_lat, midpoint_lon], zoom_start=10)

    # Legg til en tittel
    tittel_html = f'''
    <h3 align="center" style="font-size:16px"><b>Skipsbevegelser: {tidspunkt}</b></h3>
    '''
    m.get_root().html.add_child(folium.Element(tittel_html))

    # Fargekoding basert på skipstype
    ship_type_colors = {
        30: 'blue',      # Fiskefartøy
        31: 'green',     # Slepebåt
        52: 'red',       # Passasjerskip
        60: 'purple',    # Passasjerskip
        70: 'orange',    # Lasteskip
        80: 'darkblue',  # Tankskip
        # Legg til flere skipstyper etter behov
    }

    # Legg til skip på kartet
    legg_til_skip_på_kart(m, data, ship_type_colors)

    # Legg til tegnforklaring
    legg_til_tegnforklaring(m, ship_type_colors)

    # Returner kartet
    return m

def legg_til_skip_på_kart(kart, data, farger):
    """Legg til hvert skip på kartet med ikoner og retningspiler"""
    for idx, row in data.iterrows():
        # Bestem farge basert på skipstype
        color = farger.get(row['ship_type'], 'gray')

        # Hent koordinater
        coords = (row.geometry.y, row.geometry.x)

        # Lag popup-info
        popup_text = f"""
        <b>Skip:</b> {row['ship_name']}<br>
        <b>MMSI:</b> {row['mmsi']}<br>
        <b>Type:</b> {row['ship_type']}<br>
        <b>Hastighet:</b> {row['speed_over_ground']} knop<br>
        <b>Kurs:</b> {row['course_over_ground']}°<br>
        <b>Tidspunkt:</b> {row['date_time_utc']}
        """

        # Legg til markør med retningspil
        folium.Marker(
            coords,
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color=color, icon='ship', prefix='fa'),
            tooltip=f"{row['ship_name']} ({row['mmsi']})"
        ).add_to(kart)

        # Tegn retningspil hvis kurs er tilgjengelig
        if not pd.isna(row['course_over_ground']):
            folium.RegularPolygonMarker(
                coords,
                fill_color=color,
                number_of_sides=3,
                radius=8,
                rotation=row['course_over_ground'],
                fill_opacity=0.6,
                color='black',
                weight=1
            ).add_to(kart)

def legg_til_tegnforklaring(kart, farger):
    """Legg til tegnforklaring på kartet"""
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; right: 50px; width: 150px; height: 160px;
                border:2px solid grey; z-index:9999; font-size:12px;
                background-color: white; padding: 10px;
                border-radius: 5px;">
    <b>Skipstyper:</b><br>
    '''

    for ship_type, color in farger.items():
        legend_html += f'<i class="fa fa-ship" style="color:{color}"></i> Type {ship_type}<br>'

    legend_html += '</div>'
    kart.get_root().html.add_child(folium.Element(legend_html))

def hent_tidspunkt_fra_mappe(mappe_navn):
    """Hent time og minutt fra mappenavnet"""
    match = re.search(r"time_minutt=(\d{2})_(\d{2})", mappe_navn)
    if match:
        time, minutt = match.groups()
        return time, minutt
    return None, None

def simuler_datastrømming(hovedmappe, pause_sekunder=0.5, lagre_kart=False):
    """Simuler datastrømming med visualisering av skipsbevegelser"""
    # Finn alle intervaller
    alle_intervaller = finn_minutt_intervaller(hovedmappe)

    print("\nSimulerer datastrømming med visualisering...")
    for intervall_dir in alle_intervaller:
        # Hent time og minutt fra mappenavnet
        time, minutt = hent_tidspunkt_fra_mappe(intervall_dir)
        if not time or not minutt:
            continue

        tidspunkt = f"{time}:{minutt}"
        intervall_sti = os.path.join(hovedmappe, intervall_dir)

        # Vis prosesseringsinformasjon
        clear_output(wait=True)  # Fjern forrige output i notebook
        print(f"\nProsesserer data for {tidspunkt}...")

        # Les data for dette intervallet
        samlet_data, antall_filer = les_data_for_intervall(intervall_sti)

        if samlet_data is not None:
            # Beregn statistikk
            stats = beregn_statistikk(samlet_data)

            # Vis statistikker
            print(f"Kl. {tidspunkt}: Lastet {stats['totalt_antall_rader']} rader fra {antall_filer} filer")
            print(f"  Gjennomsnittlig hastighet (SOG): {stats['gjennomsnittlig_hastighet']:.2f}")
            print(f"  Antall unike skip: {stats['antall_unike_skip']}")

            # Opprett og vis kart
            kart = vis_skip_på_kart(samlet_data, f"Kl. {tidspunkt}")

            # Lagre kartet til en HTML-fil hvis ønsket
            if lagre_kart:
                kart_filnavn = f"kart_{time}_{minutt}.html"
                kart.save(kart_filnavn)
                print(f"  Kart lagret som {kart_filnavn}")

            # Vis kartet i notebook
            display(kart)
        else:
            print(f"Ingen data funnet for kl. {tidspunkt}")

        # Simuler tid mellom intervaller
        time_module.sleep(pause_sekunder)

    print("\nDatastrømming komplett!")

3. Simulering og visualisering

In [ ]:
# Eksempelbruk i notebook
# Sett inn en celle med følgende for å kjøre simuleringen:
# simuler_datastrømming(PARTISJON_MAPPE_MIN, pause_sekunder=0.5, lagre_kart=False)